In [2]:
import COVID19Py
import pandas as pd
from datetime import date,  datetime

In [3]:
covid19 = COVID19Py.COVID19()

In [4]:
def tranforma_data(date_str):
    dt = datetime.strptime(date_str, '%Y-%m-%dT%H:%M:%SZ')
    data_nova = date(dt.year, dt.month, dt.day).strftime("%d/%m/%Y") 
    return data_nova

In [11]:
def data_por_pais(pais, ps):
    
    casos_confirmados = pd.DataFrame(data = pais[0]['timelines']['confirmed'])
    # removendo coluna latest
    del casos_confirmados['latest']

# tranformado o index em coluna 
    casos_confirmados['Data'] = casos_confirmados.index

    # renomeando as colunas
    casos_confirmados.columns = ['Casos', 'Data']

    # modifica o formato da data
    casos_confirmados['Data'] = casos_confirmados['Data'].apply(tranforma_data)

    # selecioando apenas com os com valores acima de 100 casos comfimados
    selecao = casos_confirmados.Casos > 200
    casos_confirmados = casos_confirmados[selecao]

    # reiniciando o index
    casos_confirmados.index = range(casos_confirmados.shape[0])

    mortes_confirmados = pd.DataFrame(data = pais[0]['timelines']['deaths'])
    # removendo coluna latest
    del mortes_confirmados['latest']

    # tranformado o index em coluna e reiniciando o index
    mortes_confirmados['Data'] = mortes_confirmados.index

    # renomeando as colunas
    mortes_confirmados.columns = ['Mortes', 'Data']

    # modifica o formato da data
    mortes_confirmados['Data'] = mortes_confirmados['Data'].apply(tranforma_data)

    # selecionando apenas com os com valores acima de 100 casos comfimados
    mortes_confirmados = mortes_confirmados[selecao]

    # reiniciando o index
    mortes_confirmados.index = range(mortes_confirmados.shape[0])    
    
    casos_e_mortes_confirmados = pd.DataFrame([casos_confirmados.Data,
                                           casos_confirmados.Casos, 
                                           mortes_confirmados.Mortes]).transpose()
    casos_e_mortes_confirmados['Pais'] = ps
    casos_e_mortes_confirmados['Dias'] = casos_e_mortes_confirmados.index
    
    
    return casos_e_mortes_confirmados

# Obtendo os dados dos paises

## Definido os paises desejados

In [41]:
paises = [('US', 'USA'), ('BR', 'Brasil'), ('ES', 'Espanha'), ('IT', 'Itália'), ('DE', 'Alemanha') ]

## Gerando do DataFrame

In [42]:
df = pd.DataFrame()
for cd, nome in paises:
    pais = covid19.getLocationByCountryCode(cd, timelines = True)
    df_novo = data_por_pais(pais, nome)    
    df = pd.concat([df, df_novo], ignore_index = True)

casos_e_mortes_confirmados = df.copy()    

In [113]:
f = lambda x, y: x/y

s_casos = casos_e_mortes_confirmados['Casos'].values
s_mortes = casos_e_mortes_confirmados['Mortes'].values

casos_e_mortes_confirmados['Porcentagem'] = casos_e_mortes_confirmados['Mortes'] / casos_e_mortes_confirmados['Casos'] * 100

casos_e_mortes_confirmados['Porcentagem'] = (casos_e_mortes_confirmados['Porcentagem'].astype(float)).round(2)
casos_e_mortes_confirmados['Porcentagem']

0      5.53
1      5.34
2      4.23
3      4.05
4      3.77
       ... 
170    1.50
171    1.58
172    1.75
173    1.87
174    2.07
Name: Porcentagem, Length: 175, dtype: float64

## Salvando DataFrame em um arquivo csv

In [114]:
casos_e_mortes_confirmados.to_csv('casos_e_mortes_confirmados.csv', index_label = False)